In [37]:
pip install opencv-python


  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/fb/c4/f574ba6f04e6d7bf8c38d23e7a52389566dd7631fee0bcdd79ea07ef2dbf/opencv_python-4.8.0.76-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.1 MB 435.7 kB/s eta 0:01:28
   ---------------------------------------- 0.0/38.1 MB 281.8 kB/s eta 0:02:15
   ---------------------------------------- 0.1/38.1 MB 363.1 kB/s eta 0:01:45
   ---------------------------------------- 0.1/38.1 MB 326.8 kB/s eta 0:01:57
   ---------------------------------------- 0.1/38.1 MB 374.1 kB/s eta 0:01:42
   ---------------------------------------- 0.1/38.1 MB 409.6 kB/s eta 0:01:33
   ---------------------------------------- 0.1/38.1 MB 405.9 kB/s eta 0:01:34
   ---------------------------------------- 0.2/38.1 MB 399.3 kB/s eta 0:01:35
   ----

In [26]:
import os
import glob
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Add, Conv2D, BatchNormalization, Activation, AveragePooling2D, Flatten, Dense
from tensorflow.keras.initializers import random_uniform, glorot_uniform
from tensorflow.keras.models import Model
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras.layers import Input, Add, Conv2D, BatchNormalization, Activation, AveragePooling2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform, random_uniform
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Add, Conv2D, BatchNormalization, Activation, AveragePooling2D, Flatten, Dense
from tensorflow.keras.initializers import random_uniform, glorot_uniform
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix

In [56]:
# Load Landsat data (multiband geotiff) and the land cover classification map
landsat_geotiff_path = 'C:/Users/mrg22-020914479/Downloads/TestDL_2022.tif'
land_cover_map_path = 'C:/Users/mrg22-020914479/Downloads/lulc.tif'

In [57]:
# Read Landsat data using rasterio
with rasterio.open(landsat_geotiff_path) as landsat_ds:
    landsat_data = landsat_ds.read()[:5]  # Read the first 5 bands
    # Check for NaN values and replace them with a default value (e.g., -1)
landsat_data[np.isnan(landsat_data)] = 0

# Read the land cover classification map
with rasterio.open(land_cover_map_path) as land_cover_ds:
    land_cover_map = land_cover_ds.read(1)  #it's a single-band classification map
    
land_cover_map[np.isnan(land_cover_map)] = 0

In [61]:
import numpy as np
unique_values, counts = np.unique(land_cover_map, return_counts=True)
print("Unique Values:", unique_values)
print("Counts:", counts)

Unique Values: [ 0.         2.         2.142857   4.2380953  4.3333335  4.4761906
  4.571429   4.857143   4.952381   5.         5.5238094  5.6666665
  5.857143   5.904762   5.952381   6.142857   6.7619047  6.8095236
  7.095238   7.2380953  7.285714   7.6666665  7.8095236  7.952381
  8.         8.047619   8.142858   8.190476   8.619047   9.238095
  9.285714   9.333333   9.761905   9.904762  10.238095  10.476191
 11.        11.047619  11.095238  11.142858  11.190476  12.
 12.095238  12.190476  12.238095  12.285714  12.380953  12.476191
 12.571428  12.666667  12.761905  12.809524  12.857142  12.952381
 13.        13.047619  13.142858  13.190476  13.238095  13.285714
 13.333333  13.428572  13.523809  13.619047  13.714286  13.761905
 13.809524  13.904762  14.       ]
Counts: [103562   1135    195    355   2202   3078    578    162   3048    147
   1748   3762   2217   1144    751   2220   2215   4438   1969   1354
   2046   2223   2213   1248   2121   2224   2214   2214     60   1182
   155

In [82]:
# Preprocess the data
# Assuming you have a land cover class mapping (e.g., class 1 corresponds to water, class 2 to forest, etc.)
num_classes = len(np.unique(land_cover_map))
class_mapping = {1: 'Water', 2: 'Forest', 3: 'Urban', 4: 'Agriculture', 5: 'Other'}

# Prepare training data
X_train = landsat_data.transpose(1, 2, 0)  # Reshape data to (height, width, bands)
y_train = tf.keras.utils.to_categorical(land_cover_map - 1, num_classes)  # Convert to one-hot encoding

# Normalize the input data (optional but recommended)
X_train = (X_train - np.min(X_train)) / (np.max(X_train) - np.min(X_train))

# Split the dataset into training and validation sets (optional but recommended)
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)



# Print the shapes to verify
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)


X_train shape: (612, 701, 5)
y_train shape: (612, 701, 69)
X_val shape: (154, 701, 5)
y_val shape: (154, 701, 69)


In [100]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add

def identity_block(X, f, filters, training=True, initializer=random_uniform):
    """
    Implementation of the identity block as defined in Figure 4 of the ResNet paper.
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- initializer for kernel weights
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value.
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    
    return X

def convolutional_block(X, f, filters, s=2, training=True, initializer=glorot_uniform):
    """
    Implementation of the convolutional block as defined in Figure 4 of the ResNet paper.
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used
    training -- True: Behave in training mode
                False: Behave in inference mode
    initializer -- initializer for kernel weights
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)
    
    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid', kernel_initializer=initializer(seed=0))(X)
    X = BatchNormalization(axis=3)(X, training=training)

    # Shortcut path
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_initializer=initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut, training=training)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X_shortcut, X])
    X = Activation('relu')(X)
    
    return X


In [101]:
from tensorflow.keras.layers import Input, Add, Conv2D, BatchNormalization, Activation, AveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model

def ResNet50(input_shape=(612,701, 5), num_classes=69):
    # Model architecture with variable input shape
    # ...

    """
    Stage-wise implementation of the architecture of the popular ResNet50
    """
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = X_input  # No padding needed for this input shape

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), padding='same', kernel_initializer='he_normal')(X)
    X = BatchNormalization()(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL
    X = AveragePooling2D(pool_size=(2, 2))(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(num_classes, activation='softmax', kernel_initializer='he_normal')(X)

    # Create model
    model = Model(inputs=X_input, outputs=X)

    return model


In [103]:
# Define the ResNet50 model using the ResNet50 function
model = ResNet50(input_shape=(64, 64, 5), num_classes=6)

# Compile the Model
model.compile(
    optimizer='adam',  # You can choose another optimizer if needed (e.g., 'sgd', 'rmsprop')
    loss='categorical_crossentropy',  # Use categorical cross-entropy for multi-class classification
    metrics=['accuracy']  # You can add more metrics for evaluation if desired
)


In [104]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20


ValueError: in user code:

    File "C:\Users\mrg22-020914479\.conda\envs\TestDL\Lib\site-packages\keras\src\engine\training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\mrg22-020914479\.conda\envs\TestDL\Lib\site-packages\keras\src\engine\training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\mrg22-020914479\.conda\envs\TestDL\Lib\site-packages\keras\src\engine\training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\mrg22-020914479\.conda\envs\TestDL\Lib\site-packages\keras\src\engine\training.py", line 1080, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\mrg22-020914479\.conda\envs\TestDL\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\mrg22-020914479\.conda\envs\TestDL\Lib\